In [1]:
import sparknlp
# spark = sparknlp.start() 
spark = sparknlp.start(apple_silicon=True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import *
import pyspark
print(f"Spark NLP version: {sparknlp.version()}")
print(f"Spark version: {pyspark.__version__}")



:: loading settings :: url = jar:file:/Users/dannytram/MYSELF/MLOps_K6/final_project/Livestream-Assistant-Agent/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/dannytram/.ivy2/cache
The jars for the packages stored in: /Users/dannytram/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-silicon_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bcd12085-f586-4395-86c8-84be576e1e09;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-silicon_2.12;6.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath

25/12/16 15:57:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version: 6.2.3
Spark version: 3.3.1


In [2]:
MODEL_NAME = "sentimentdl_use_twitter"

documentAssembler = DocumentAssembler()\
    .setInputCol("reviewText")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
    stages = [
        documentAssembler,
        use,
        sentimentdl
        ])

print("Pipeline defined successfully!")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]25/12/16 15:57:36 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/12/16 15:57:37 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ / ]Download done! Loading the resource.
[ \ ]

2025-12-16 15:57:45.054648: W external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


[ | ]

[OK!]
sentimentdl_use_twitter download started this may take some time.
25/12/16 15:57:49 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
Approximate size to download 11.4 MB
[ | ]25/12/16 15:57:50 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/12/16 15:57:51 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
sentimentdl_use_twitter download started this may take 

In [ ]:
print("\nFitting pipeline structure (no training involved)...")
empty_df = spark.createDataFrame([[""]], ["reviewText"])

pipelineModel = nlpPipeline.fit(empty_df)
print("Pipeline ready for inference!")


Fitting pipeline structure (no training involved)...
Pipeline ready for inference!


In [11]:
# Wrap your trained model
light_pipeline = LightPipeline(pipelineModel)

# Instant inference on a simple string (no Spark overhead)
print(light_pipeline.annotate("fantastic"))

{'document': ['fantastic'], 'sentence_embeddings': ['fantastic'], 'sentiment': ['positive']}
